In [8]:
import pandas as pd
import os
import numpy as np
import psycopg2
import glob
import openpyxl

In [9]:
df = pd.read_excel("Invoice.xlsx")
df.head()

,Invoice Date,Invoice ID,Invoice Number,Invoice Status,Customer ID,Customer Name,Is Inclusive Tax,Due Date,Currency Code,Exchange Rate,...,Tax ID,Item Tax,Item Tax Amount,Item Tax Type,CF.Origin Country,CF.Destination Country,CF.Cargo Value,CF.Shipment type,CF.Item Description,CF.Service type
0,2021-01-05,2620362000001135101,INV-000302,Closed,2620362000000078039,VLEC Delivery Services - GHS,False,2021-01-05,GHS,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-08,2620362000001135131,INV-000305,Closed,2620362000000082027,Organic Pavillion - USD,False,2021-01-08,USD,5.860016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-11,2620362000001135145,INV-000312,Void,2620362000000082412,Hans & Ezra - USD,False,2021-01-11,USD,5.859570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-11,2620362000001135145,INV-000312,Void,2620362000000082412,Hans & Ezra - USD,False,2021-01-11,USD,5.859570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-11,2620362000001135145,INV-000312,Void,2620362000000082412,Hans & Ezra - USD,False,2021-01-11,USD,5.859570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.columns = [x.lower().replace(" ","_").replace("%","").replace("&","").replace(".","") for x in df.columns]
df.columns

Index(['invoice_date', 'invoice_id', 'invoice_number', 'invoice_status',
       'customer_id', 'customer_name', 'is_inclusive_tax', 'due_date',
       'currency_code', 'exchange_rate', 'discount_type',
       'is_discount_before_tax', 'template_name', 'subtotal', 'total',
       'balance', 'adjustment', 'adjustment_description',
       'expected_payment_date', 'last_payment_date', 'payment_terms',
       'payment_terms_label', 'notes', 'terms__conditions',
       'entity_discount_amount', 'branch_id', 'branch_name', 'shipping_charge',
       'item_name', 'item_desc', 'quantity', 'discount', 'discount_amount',
       'item_total', 'usage_unit', 'product_id', 'sales_order_number',
       'expense_reference_id', 'round_off', 'sales_person', 'subject',
       'primary_contact_emailid', 'primary_contact_mobile',
       'primary_contact_phone', 'estimate_number', 'account', 'account_code',
       'tax_id', 'item_tax', 'item_tax_amount', 'item_tax_type',
       'cforigin_country', 'cfdestinat

In [11]:
df.dtypes

invoice_date                object
invoice_id                   int64
invoice_number              object
invoice_status              object
customer_id                  int64
customer_name               object
is_inclusive_tax              bool
due_date                    object
currency_code               object
exchange_rate              float64
discount_type               object
is_discount_before_tax        bool
template_name               object
subtotal                   float64
total                      float64
balance                    float64
adjustment                   int64
adjustment_description      object
expected_payment_date      float64
last_payment_date           object
payment_terms                int64
payment_terms_label         object
notes                       object
terms__conditions           object
entity_discount_amount     float64
branch_id                    int64
branch_name                 object
shipping_charge              int64
item_name           

In [12]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'varchar',
    'datetime64[ns]': 'varchar',
    'datetime64': 'timestamp',
    'bool' : 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'varchar',
 'datetime64[ns]': 'varchar',
 'datetime64': 'timestamp',
 'bool': 'varchar'}

In [13]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'invoice_date varchar, invoice_id varchar, invoice_number varchar, invoice_status varchar, customer_id varchar, customer_name varchar, is_inclusive_tax varchar, due_date varchar, currency_code varchar, exchange_rate float, discount_type varchar, is_discount_before_tax varchar, template_name varchar, subtotal float, total float, balance float, adjustment varchar, adjustment_description varchar, expected_payment_date float, last_payment_date varchar, payment_terms varchar, payment_terms_label varchar, notes varchar, terms__conditions varchar, entity_discount_amount float, branch_id varchar, branch_name varchar, shipping_charge varchar, item_name varchar, item_desc varchar, quantity float, discount float, discount_amount float, item_total float, usage_unit float, product_id float, sales_order_number float, expense_reference_id float, round_off float, sales_person varchar, subject varchar, primary_contact_emailid varchar, primary_contact_mobile varchar, primary_contact_phone varchar, estim

In [33]:
#open database connection


conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [34]:
cursor.execute("drop table if exists Invoices;")

In [35]:
cursor.execute("create table Invoices \
    (invoice_date date, invoice_id varchar, invoice_number varchar, invoice_status varchar, customer_id varchar, customer_name varchar, is_inclusive_tax varchar, due_date date, currency_code varchar, exchange_rate float, discount_type varchar, is_discount_before_tax varchar, template_name varchar, subtotal float, total float, balance float, adjustment varchar, adjustment_description varchar, expected_payment_date date, last_payment_date date, payment_terms varchar, payment_terms_label varchar, notes varchar, terms__conditions varchar, entity_discount_amount float, branch_id varchar, branch_name varchar, shipping_charge varchar, item_name varchar, item_desc varchar, quantity float, discount float, discount_amount float, item_total float, usage_unit float, product_id float, sales_order_number float, expense_reference_id float, round_off float, sales_person varchar, subject varchar, primary_contact_emailid varchar, primary_contact_mobile varchar, primary_contact_phone varchar, estimate_number varchar, account varchar, account_code varchar, tax_id float, item_tax varchar, item_tax_amount float, item_tax_type varchar, cforigin_country varchar, cfdestination_country varchar, cfcargo_value float, cfshipment_type varchar, cfitem_description varchar, cfservice_type varchar)")

In [36]:
#insert values to tables
#save df to csv
df.to_csv('Invoices', header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object and upload to db
my_file = open("Invoices")
print('file opened in memory')

file opened in memory


In [38]:
SQL_STATEMENT = """
COPY Invoices FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [39]:
cursor.execute("grant select on table Invoices to public")
conn.commit()

cursor.close()
print('table invoices made imported successfully')

table invoices made imported successfully
